In [ ]:
!pip install -q transformers
!pip install -q simpletransformers

     |████████████████████████████████| 3.8 MB 3.9 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 
     |████████████████████████████████| 6.5 MB 35.6 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 248 kB 4.0 MB/s 
     |████████████████████████████████| 1.7 MB 48.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 9.9 MB 18.0 MB/s 
     |████████████████████████████████| 1.2 MB 44.9 MB/s 
     |████████████████████████████████| 312 kB 53.8 MB/s 
     |████████████████████████████████| 181 kB 48.1 MB/s 
     |████████████████████████████████| 144 kB 39.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 134 kB 52.2 MB/s 
     |████████████████████████████████| 212 kB 41.1 MB/s 
     |████████████████

In [ ]:
import pandas as pd
import csv
import urllib.request
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

In [ ]:


def filtered(sentence):
  words = sentence.split()
  filter = ['sarcasm', 'sarcastic', 'reposting', '<url>', 'joke', 'humour', 'humor', 'jokes', 'irony', 'ironic', 'exgag']
  for filtered_word in filter:
    if filtered_word in words:
      return True
  return False


def download_and_clean(url, output_file, text_index, labels_index, to_filter=False):
  with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['text', 'labels'])
    file = urllib.request.urlopen(url)
    for line in file:
      decoded_line = line.decode('utf-8')
      row = eval(decoded_line)
      if not to_filter or not filtered(row[text_index]):
        csv_writer.writerow([row[text_index], row[labels_index]])

In [ ]:
download_and_clean('https://raw.githubusercontent.com/headacheboy/data-of-multimodal-sarcasm-detection/master/text/train.txt', 'train.csv', 1, 2, to_filter=True)
download_and_clean('https://raw.githubusercontent.com/headacheboy/data-of-multimodal-sarcasm-detection/master/text/test2.txt', 'test.csv', 1, 3)
download_and_clean('https://raw.githubusercontent.com/headacheboy/data-of-multimodal-sarcasm-detection/master/text/valid2.txt', 'validate.csv', 1, 3)

In [ ]:
tr = pd.read_csv('train.csv')
te = pd.read_csv('test.csv')
val = pd.read_csv('validate.csv')
tr.head()

,text,labels
0,<user> thanks for showing up for our appointme...,1
1,haha . # lol,1
2,i love waiting <num> min for a cab - such shor...,1
3,22 super funny quotes # funnyquotes # funnysa...,1
4,goog morning # sorrynotsorry # morning,1


In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': False,
    'max_seq_length': 75,
    'learning_rate': 0.00005,
    'weight_decay': 0.01,
    'warmup_ratio': 0.2,
    'max_grad_norm': 1.0,
    'num_train_epochs': 3,
    'train_batch_size': 32,
    'save_model_every_epoch': False,
    'save_steps': 4000,
    'fp16': True,
    'evaluate_during_training': True,
}

In [ ]:
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args) 

model.train_model(tr, eval_df=val)

result, model_outputs, wrong_predictions = model.eval_model(te, acc=sklearn.metrics.accuracy_score)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139762322024912 on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:filelock:Lock 139762322024912 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /roberta-base/resolve/main/config.json HTTP/1.1" 200 481


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139762322024912 on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:filelock:Lock 139762322024912 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 139762322076112 on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
DEBUG:filelock:Lock 139762322076112 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139762322076112 on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
DEBUG:filelock:Lock 139762322076112 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_75_2_2
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2409 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_75_2_2


Running Evaluation:   0%|          | 0/302 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8751351185548726, 'tp': 908, 'tn': 1355, 'fp': 95, 'fn': 51, 'auroc': 0.9443008162237962, 'auprc': 0.8888400862593071, 'eval_loss': 0.5393895492549764, 'acc': 0.9393939393939394}


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(te, acc=sklearn.metrics.f1_score)
model_outputs

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2409 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_75_2_2


Running Evaluation:   0%|          | 0/302 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8751351185548726, 'tp': 908, 'tn': 1355, 'fp': 95, 'fn': 51, 'auroc': 0.9443008162237962, 'auprc': 0.8888400862593071, 'eval_loss': 0.5393895492549764, 'acc': 0.9255861365953109}


array([[-4.48046875,  4.4296875 ],
       [-4.4609375 ,  4.4453125 ],
       [-4.4765625 ,  4.4453125 ],
       ...,
       [ 4.68359375, -4.4296875 ],
       [ 4.73046875, -4.52734375],
       [ 4.546875  , -4.32421875]])

In [ ]:
samples = ["I totally understand man! 😏",'this routine really suits us! # silly # funny # relatable']
predictions, _ = model.predict(samples) 
label_dict = {0: 'not sarcastic', 1: 'sarcastic'}
ff = ['🙂','😂','🤣','😌','😒','😏','💩','🤭','👀','🍌','🍆','🍒','🍑','🚩','🌝','🌚']
for idx, sample in enumerate(samples):
  f=0
  for j in ff:
    if j in sample:
      print('{}: {}, {}'.format(idx, sample, 'sarcastic'))
      f=1
  if f==0:
    print('{}: {}, {}'.format(idx, sample, label_dict[predictions[idx]]))


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0: I totally understand man! 😏, sarcastic
1: this routine really suits us! # silly # funny # relatable, not sarcastic


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
model_save_name = 'sarcasm.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model, path)